# Improve the Model (Version 2)
 🔧 What We’ll Do:
- Make the model deeper: More Conv1D + BatchNorm + Dropout
- Use Bidirectional LSTM to better capture emotion flow
- Stratified split so all emotion classes are balanced

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

Load features

In [11]:
X = np.load("features.npy")
y = np.load("labels.npy")

Encode labels

In [12]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

Train-test split with stratify

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)

Build better model

In [14]:
model = Sequential([
    Input(shape=(40, 173)),
    Conv1D(64, 5, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),
    Conv1D(128, 5, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.1637 - loss: 2.0899 - val_accuracy: 0.1572 - val_loss: 2.0303
Epoch 2/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1978 - loss: 2.0138 - val_accuracy: 0.2448 - val_loss: 1.9299
Epoch 3/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2483 - loss: 1.9253 - val_accuracy: 0.3479 - val_loss: 1.8046
Epoch 4/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2903 - loss: 1.8036 - val_accuracy: 0.3505 - val_loss: 1.7535
Epoch 5/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3011 - loss: 1.7922 - val_accuracy: 0.3918 - val_loss: 1.6637
Epoch 6/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3532 - loss: 1.7197 - val_accuracy: 0.3943 - val_loss: 1.5987
Epoch 7/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.3814 - loss: 1.6117 - val_accuracy: 0.4201 - val_loss: 1.5138
Epoch 8/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4252 - loss: 1.5361 - val_accuracy: 0.4665 - v

In [17]:
model.save("emotion_model_v3.keras")

In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Improved Test accuracy:", test_acc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6221 - loss: 1.5442
Improved Test accuracy: 0.6185566782951355
